# Week 3: Segmenting and Clustering Neighborhoods in Toronto

### 1. Loading libraries

In [1]:
# loading libraries from Lab's notebook

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#!conda install -c beautifulsoup4
from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


### 2. Scrapping list of neighborhoods from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [2]:
r  = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
data = r.text
soup = BeautifulSoup(data)
soup = BeautifulSoup(str(soup).replace("\n", ""))

In [3]:
Postal = []
Borough = []
Neighbourhood = []

for items in soup.find('table', class_='wikitable').find_all('tr')[1::1]:
    data = items.find_all(['th','td'])
    try:
        Postal.append(data[0].text)
        Borough.append(data[1].text)
        Neighbourhood.append(data[2].text)
    except IndexError:pass


neighbourhood = pd.DataFrame(pd.DataFrame(data={
    'Postal' : Postal,
    'Borough' : Borough,
    'Neighborhood' : Neighbourhood
    }))

### 3. Cleaning and organizing the data

In [4]:
neighbourhood['Neighborhood'].replace("Not assigned", neighbourhood["Borough"],inplace=True)
neighbourhood = neighbourhood[neighbourhood.Borough != 'Not assigned']
neighbourhood = neighbourhood.groupby(['Postal', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()

### 4. Printing head and shape of dataframe

In [5]:
neighbourhood.head(10)

,Postal,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"
